In [5]:
pip install finvizfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00


In [7]:
from finvizfinance.screener.overview import Overview
import pandas as pd
import numpy as np

def get_mid_cap_ram():
    # Obtener stocks de Finviz
    screener = Overview()
    filters_dict = {
        'Market Cap.': '+Mid (over $2bln)',
        'Average Volume': 'Over 500K',
        'Price': 'Over $10'
    }

    screener.set_filter(filters_dict=filters_dict)
    stocks_df = screener.screener_view()

    # Calcular RAM para cada stock
    ram_scores = {}
    for idx, row in stocks_df.iterrows():
        ticker = row['Ticker']
        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(period="2y")

            # Skip si no hay suficientes datos
            if len(hist) < 252:
                continue

            returns = hist['Close'].pct_change()
            realized_vol = returns.rolling(window=63).std() * np.sqrt(252)
            vol_adj_returns = returns / realized_vol
            ram_score = vol_adj_returns.rolling(window=252).mean().iloc[-1]

            ram_scores[ticker] = {
                'RAM_Score': ram_score,
                'Company': row['Company'],
                'Sector': row['Sector'],
                'Market_Cap': row['Market Cap']
            }

        except Exception as e:
            print(f"Error processing {ticker}: {str(e)}")
            continue

    # Convertir a DataFrame y ordenar por RAM score
    ram_df = pd.DataFrame.from_dict(ram_scores, orient='index')
    ram_df = ram_df.sort_values('RAM_Score', ascending=False)

    return ram_df.head(20)

# Ejecutar análisis
top_ram = get_mid_cap_ram()
print("\nTop 20 Stocks por RAM Score:")
print(top_ram[['RAM_Score', 'Company', 'Sector']])

ERROR:yfinance:AHR: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']
ERROR:yfinance:ALAB: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']
ERROR:yfinance:AMTM: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:AS: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']
ERROR:yfinance:BTSG: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']
ERROR:yfinance:CGON: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']
ERROR:yfinance:CON: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:CORZ: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']
ERROR:yfinance:CURB: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'ma


Top 20 Stocks por RAM Score:
      RAM_Score                        Company                  Sector
TRGP   0.016136           Targa Resources Corp                  Energy
VST    0.015848                    Vistra Corp               Utilities
DTM    0.015514               DT Midstream Inc                  Energy
FTAI   0.014384              FTAI Aviation Ltd             Industrials
APP    0.012908                  Applovin Corp              Technology
GGAL   0.012817   Grupo Financiero Galicia ADR               Financial
PRMB   0.012710              Primo Brands Corp      Consumer Defensive
SFM    0.012702     Sprouts Farmers Market Inc      Consumer Defensive
WMT    0.012513                    Walmart Inc      Consumer Defensive
BBAR   0.012456             BBVA Argentina ADR               Financial
EAT    0.012374     Brinker International, Inc       Consumer Cyclical
MPLX   0.011908                        MPLX LP                  Energy
IBKR   0.011690  Interactive Brokers Group Inc 